# PostgreSQL Connection Example

This notebook demonstrates how to connect to a PostgreSQL database using PandasAI v3.
The `pai.create()` function is used to create a DataFrame from a PostgreSQL database connection.

In [ ]:
# Install pandasai-sql[postgres] if not already installed:
!pip install 'pandasai-sql[postgres]'


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
# Setup PostgreSQL database with sample data
import psycopg2
from psycopg2 import sql

# Database connection parameters
db_config = {
    "host": "localhost",
    "port": 5432,
    "user": "demo",
    "password": "secret",
    "database": "revops",
}

# SQL script to create table and insert data
setup_sql = """
-- Sample data for analytics-db PostgreSQL database
-- This script creates a sample revenue table with region data

-- Create a revenue table
CREATE TABLE IF NOT EXISTS revenue (
    id SERIAL PRIMARY KEY,
    region VARCHAR(100) NOT NULL,
    amount DECIMAL(10, 2) NOT NULL,
    date DATE NOT NULL,
    product VARCHAR(100)
);

-- Insert sample data
INSERT INTO revenue (region, amount, date, product) VALUES
    ('North America', 150000.00, '2024-01-15', 'Product A'),
    ('North America', 175000.00, '2024-02-20', 'Product B'),
    ('Europe', 120000.00, '2024-01-10', 'Product A'),
    ('Europe', 135000.00, '2024-02-15', 'Product C'),
    ('Asia Pacific', 95000.00, '2024-01-25', 'Product A'),
    ('Asia Pacific', 110000.00, '2024-02-28', 'Product B'),
    ('South America', 75000.00, '2024-01-05', 'Product C'),
    ('South America', 85000.00, '2024-02-10', 'Product A'),
    ('North America', 160000.00, '2024-03-12', 'Product C'),
    ('Europe', 140000.00, '2024-03-18', 'Product B'),
    ('Asia Pacific', 105000.00, '2024-03-22', 'Product C'),
    ('South America', 80000.00, '2024-03-25', 'Product B')
ON CONFLICT DO NOTHING;
"""

try:
    # Connect to PostgreSQL
    conn = psycopg2.connect(**db_config)
    conn.autocommit = True
    cursor = conn.cursor()
    
    # Execute the setup SQL script
    cursor.execute(setup_sql)
    
    # Verify the table was created and data was inserted
    cursor.execute("SELECT COUNT(*) FROM revenue;")
    count = cursor.fetchone()[0]
    
    print(f"✓ Database setup completed successfully!")
    print(f"✓ Table 'revenue' created with {count} rows")
    
    cursor.close()
    conn.close()
    
except psycopg2.Error as e:
    print(f"Error setting up database: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")


In [ ]:
from pandasai_litellm.litellm import LiteLLM
import pandasai as pai
import pandas as pd

import os
# 1. Configure LLM and PandasAI
api_key = os.getenv("OPENAI_API_KEY", "your-api-key")
llm = LiteLLM(model="gpt-5-mini", api_key=api_key)

pai.config.set({
    "llm": llm,
    "save_logs": True,
    "max_retries": 3
})


df = pai.create(
    path="demo/revenue",
    description="Revenue data from PostgreSQL database",
    source={
        "type": "postgres",
        "connection": {
            "host": "localhost",
            "port": 5432,
            "user": "demo",
            "password": "secret",
            "database": "revops",
        },
        "table": "revenue",
    }
)

# 3. Use DataFrame to chat with the data
response = df.chat("Show me the total revenue by region")
print(response)


Dataset saved successfully to path: demo/revenue3
          region  total_revenue
0  North America       485000.0
1         Europe       395000.0
2   Asia Pacific       310000.0
3  South America       240000.0
